# NLP 2 과제
> 인공지능 스터디 일곱 번째 과제에 오신 것을 환영합니다! 강의를 들으면서 배운 다양한 지식들을 실습을 통해서 활용해 볼 시간을 가질 것입니다!

#### ❓ <font color='red'><b>[ 퀴즈 ]</b></font> Transformer

아래의 수식과 같이 계산되는 multi-head attention에서 query, key, value 벡터를 생성하기 위한
projection matrix $( W_{i}^{Q}, W_{i}^{K}, W_{i}^{V}​​ )$는 head 간에 sharing 된다. <br>
***
$ MultiHead(Q,K,V)=Concat(head_1, \cdots, head_h)W^{O} $ (이때, $W^{O}$ 는 Output을 만들때 사용되는 가중치 행렬)<br>
where $head_i=Attention(QW^{Q}_{i}, KW^{K}_{i}, VW^{V}_{i})$ (이때, $Q, K, V$ 는 입력에서 tokenize된 단어들의 임베딩 벡터 $Q = K = V$ )
```python
(1) 예
(2) 아니오
```

```python
😉
# TODO : 정답을 적어주세요
2
```

#### ❓ <font color='red'><b>[ 퀴즈 ]</b></font> Transformer
```python
Transformer 모델에서 각 입력 토큰들이 가진 순서를 입력하기 위해 사용하는 방법을 고르시오.

(1) Positional Encoding
(2) Encoder-Decoder attention
(3) Layer normalization
(4) Masked decoder self-attention
```

```python
😉
# TODO : 정답을 적어주세요
1
```

#### ❓ <font color='red'><b>[ 퀴즈 ]</b></font> GPT
```python
GPT-1 모델이 어떻게 다양한 자연어 처리 태스크에서 사용될 수 있는지 설명해주세요.
```

```python
😉
# TODO : 정답을 적어주세요
논리적 함의 추측, 유사성 검사, 다지선다문제 정답 선택
```

#### ❓ <font color='red'><b>[ 퀴즈 ]</b></font> GPT
```python
GPT-1 모델의 "GPT" 약자는 무엇을 의미하나요?

(1) Generalized Pre-trained Transformer
(2) Generative Pre-trained Transformer
(3) Globalized Pre-processing Transformer
(4) Gradient Propagation Technique
```

```python
😉
# TODO : 정답을 적어주세요
2
```

#### ❓ <font color='red'><b>[ 퀴즈 ]</b></font> BERT
```python
다음 중 BERT에 대한 설명으로 옳지 않은 것을 고르시오.

(1) 학습 데이터에서 [MASK] 토큰이 선택되는 비율이 극단적으로 작은 경우, 모델 학습을 위한 비용이 증가한다.
(2) Unidirectional model로 자연어 생성에 특화된 모델이다.
(3) 입력 시퀀스 중 일부 마스킹된 토큰을 맞추는 masked language modeling (masked LM)을 통해 pre-training을 수행하였다.
(4) 사전학습을 위한 [MASK] 토큰은 random하게 선택된다.
(5) Unlabeled 데이터를 기반으로 self-supervised learning을 적용하여 사전학습한 모델이다.
```


```python
😉
# TODO : 정답을 적어주세요
2
```

#### 👨‍💻 <font color='green'><b>[ 실습 ]</b></font> Multi-head Attention
```python
이번 실습을 통해 다음 2가지를 알아볼 것입니다.
1. Multi-head attention 및 self-attention을 구현합니다.
2. 각 과정에서 일어나는 연산과 input/output 형태를 이해합니다.
```

```python
🐙
먼저 코드 실행에 필요한 패키지를 import 해봅시다.
```

In [1]:
from torch import nn
from torch.nn import functional as F
from tqdm import tqdm

import torch
import math

### 데이터 전처리
```python
저번 주차의 데이터와 비슷한 형태입니다.
먼저 전체 단어 수인 vocab_size가 주어집니다.
pad_id는 주어진 데이터의 길이를 맞춰주기 위해 패딩을 진행하게 되는데 이때 패딩을 의미하는 토큰의 id입니다.
sample data 보면 숫자로 이루어진 것을 볼 수 있는데 이는 저희가 구성한 vocab에서 몇 번째 단어인지를 의미합니다.
따라서 데이터의 각 요소를 단어로 이루어진 문장이라고 생각할 수 있습니다.
```

In [2]:
vocab_size = 100
pad_id = 0

data = [
  [62, 13, 47, 39, 78, 33, 56, 13, 39, 29, 44, 86, 71, 36, 18, 75],
  [60, 96, 51, 32, 90],
  [35, 45, 48, 65, 91, 99, 92, 10, 3, 21, 54],
  [75, 51],
  [66, 88, 98, 47],
  [21, 39, 10, 64, 21],
  [98],
  [77, 65, 51, 77, 19, 15, 35, 19, 23, 97, 50, 46, 53, 42, 45, 91, 66, 3, 43, 10],
  [70, 64, 98, 25, 99, 53, 4, 13, 69, 62, 66, 76, 15, 75, 45, 34],
  [20, 64, 81, 35, 76, 85, 1, 62, 8, 45, 99, 77, 19, 43]
]

```python
주어진 데이터의 길이를 맞춰주기 위한 padding 함수를 도입합니다.
```

In [3]:
def padding(data):
  max_len = len(max(data, key=len))
  print(f"Maximum sequence length: {max_len}")

  for i, seq in enumerate(tqdm(data)):
    if len(seq) < max_len:
      data[i] = seq + [pad_id] * (max_len - len(seq))

  return data, max_len

In [4]:
data, max_len = padding(data)

Maximum sequence length: 20


100%|██████████| 10/10 [00:00<00:00, 62508.26it/s]


```python
전처리된 데이터를 확인해 보면 잘 패딩 되었음을 확인할 수 있습니다.
```

In [6]:
data

[[62, 13, 47, 39, 78, 33, 56, 13, 39, 29, 44, 86, 71, 36, 18, 75, 0, 0, 0, 0],
 [60, 96, 51, 32, 90, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [35, 45, 48, 65, 91, 99, 92, 10, 3, 21, 54, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [75, 51, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [66, 88, 98, 47, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [21, 39, 10, 64, 21, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [98, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [77,
  65,
  51,
  77,
  19,
  15,
  35,
  19,
  23,
  97,
  50,
  46,
  53,
  42,
  45,
  91,
  66,
  3,
  43,
  10],
 [70, 64, 98, 25, 99, 53, 4, 13, 69, 62, 66, 76, 15, 75, 45, 34, 0, 0, 0, 0],
 [20, 64, 81, 35, 76, 85, 1, 62, 8, 45, 99, 77, 19, 43, 0, 0, 0, 0, 0, 0]]

### Hyperparameter 세팅 및 embedding
```python
위 데이터를 임베딩하여 실습에 사용할 데이터를 만들어 봅시다.
```

In [7]:
d_model = 512  # model의 hidden size
num_heads = 8  # multi-head에서의 head의 개수

In [8]:
embedding = nn.Embedding(vocab_size, d_model)

# B: 배치 사이즈, L: maximum sequence length
batch = torch.LongTensor(data)  # (B, L)
batch_emb = embedding(batch)  # (B, L, d_model)

In [9]:
print(batch_emb)
print(batch_emb.shape)

tensor([[[-0.4282,  0.6188,  0.8736,  ..., -0.8371, -0.5100,  0.7989],
         [-1.9735,  0.2280,  0.0281,  ...,  2.0192,  0.1319,  0.0241],
         [ 0.7195,  0.2482,  1.1721,  ...,  1.1802,  0.2993,  1.6275],
         ...,
         [ 0.2835,  0.4679,  0.8529,  ..., -1.3343, -1.2773, -0.9408],
         [ 0.2835,  0.4679,  0.8529,  ..., -1.3343, -1.2773, -0.9408],
         [ 0.2835,  0.4679,  0.8529,  ..., -1.3343, -1.2773, -0.9408]],

        [[-2.1553,  1.5210,  0.4982,  ..., -1.4450,  2.2981, -0.1059],
         [-0.7475, -0.3579, -0.2764,  ...,  0.5888, -1.6453,  1.4642],
         [ 0.3176, -1.8282,  1.1573,  ..., -0.2372, -1.3562, -0.9574],
         ...,
         [ 0.2835,  0.4679,  0.8529,  ..., -1.3343, -1.2773, -0.9408],
         [ 0.2835,  0.4679,  0.8529,  ..., -1.3343, -1.2773, -0.9408],
         [ 0.2835,  0.4679,  0.8529,  ..., -1.3343, -1.2773, -0.9408]],

        [[-1.3934, -0.3462, -0.8814,  ...,  1.1212, -1.7532,  0.0119],
         [ 0.4660,  0.4771,  0.2235,  ...,  0

### Linear transformation & 여러 head로 나누기
```python
Multi-head attention 내에서 쓰이는 linear transformation matrix들을 정의합니다.

query, key, value를 서로 다른 linear transformation matrix로 행렬 연산을 통해 만들어 냅니다. 따라서 동일한 데이터(batch_emb)로부터 서로 다른 query, key, value를 생성할 수 있습니다.
```

In [10]:
w_q = nn.Linear(d_model, d_model)
w_k = nn.Linear(d_model, d_model)
w_v = nn.Linear(d_model, d_model)

```python
output layer에서 사용될 행렬도 만들어 줍니다.
```

In [11]:
w_0 = nn.Linear(d_model, d_model)

In [12]:
q = w_q(batch_emb)  # (B, L, d_model)
k = w_k(batch_emb)  # (B, L, d_model)
v = w_v(batch_emb)  # (B, L, d_model)

print(q.shape)
print(k.shape)
print(v.shape)

torch.Size([10, 20, 512])
torch.Size([10, 20, 512])
torch.Size([10, 20, 512])


```python
q, k, v를 'num_head' 개의 차원으로 분할하여 여러 벡터를 만듭니다.
실제 q, k, v 각각의 벡터 크기는 512가 아닌 64입니다.
```

In [13]:
batch_size = q.shape[0]
d_k = d_model // num_heads # q, k, v 벡터 사이즈

q = q.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)
k = k.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)
v = v.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)

print(q.shape)
print(k.shape)
print(v.shape)

torch.Size([10, 20, 8, 64])
torch.Size([10, 20, 8, 64])
torch.Size([10, 20, 8, 64])


```python
8개의 head에 필요한 q, k, v가 만들어졌습니다.
```

In [14]:
q = q.transpose(1, 2)  # (B, num_heads, L, d_k)
k = k.transpose(1, 2)  # (B, num_heads, L, d_k)
v = v.transpose(1, 2)  # (B, num_heads, L, d_k)

print(q.shape)
print(k.shape)
print(v.shape)

torch.Size([10, 8, 20, 64])
torch.Size([10, 8, 20, 64])
torch.Size([10, 8, 20, 64])


### Scaled dot-product self-attention 구현
```python
각 head에서 실행되는 self-attention 과정을 살펴봅시다.

q, k 벡터의 내적 연산 이후에 d_k의 제곱근으로 나눠줍니다.
이는 q와 k를 구성하는 요소의 평균과 분산을 내적의 결괏값에 대해서도 유지시켜주기 위함입니다.

이후 계산된 각 행에 대해서 softmax 연산을 통해서 각 요소의 합을 1로 만들어줍니다.
```
- [Scaled Dot-Product Attention 참고](https://paperswithcode.com/method/scaled)


In [15]:
attn_scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d_k)  # (B, num_heads, L, L)
attn_dists = F.softmax(attn_scores, dim=-1)  # (B, num_heads, L, L)

print(attn_dists)
print(attn_dists.shape)

tensor([[[[0.0330, 0.0564, 0.0388,  ..., 0.0611, 0.0611, 0.0611],
          [0.0461, 0.0785, 0.0470,  ..., 0.0455, 0.0455, 0.0455],
          [0.0649, 0.0714, 0.0281,  ..., 0.0499, 0.0499, 0.0499],
          ...,
          [0.0378, 0.0415, 0.0701,  ..., 0.0502, 0.0502, 0.0502],
          [0.0378, 0.0415, 0.0701,  ..., 0.0502, 0.0502, 0.0502],
          [0.0378, 0.0415, 0.0701,  ..., 0.0502, 0.0502, 0.0502]],

         [[0.0437, 0.0439, 0.0536,  ..., 0.0466, 0.0466, 0.0466],
          [0.0360, 0.0788, 0.0283,  ..., 0.0508, 0.0508, 0.0508],
          [0.0556, 0.0511, 0.0556,  ..., 0.0346, 0.0346, 0.0346],
          ...,
          [0.0448, 0.0458, 0.0502,  ..., 0.0599, 0.0599, 0.0599],
          [0.0448, 0.0458, 0.0502,  ..., 0.0599, 0.0599, 0.0599],
          [0.0448, 0.0458, 0.0502,  ..., 0.0599, 0.0599, 0.0599]],

         [[0.0294, 0.0393, 0.0628,  ..., 0.0620, 0.0620, 0.0620],
          [0.0606, 0.0325, 0.0830,  ..., 0.0384, 0.0384, 0.0384],
          [0.0884, 0.0367, 0.0656,  ..., 0

```python
이후 계산된 attention 값을 v과 곱하여 최종 결괏값을 제시합니다.
```

In [16]:
attn_values = torch.matmul(attn_dists, v)  # (B, num_heads, L, d_k)

print(attn_values.shape)

torch.Size([10, 8, 20, 64])


### 각 head의 결과 병합(concat)
```python
각 head의 결과물을 concat하고 동일 차원(d_model)으로 linear transformation 합니다.

여기서 'd_model' 차원으로 linear transformation 하는 이유는 transformer 모델에서 원래의 데이터와 더하는 연산(residual connection)이 존재하여 이때 차원을 통일해야 하기 때문입니다.

residual connection 연산은 아래 이미지에서 Self-Attention 블록 이후 Add에 해당하는 연산입니다.

residual connection은 앞선 강의에서 배운 resnet에서 소개된 기술입니다.
```
![residual](https://github.com/Pjunn/GDSC_mlstudy/blob/main/7%EC%A3%BC%EC%B0%A8/transformer_resideual_layer_norm.png?raw=true)
이미지 출처: https://jalammar.github.io/illustrated-transformer/ <br><br>
-[What is Residual Connection?](https://paperswithcode.com/method/residual-connection)

In [17]:
attn_values = attn_values.transpose(1, 2)  # (B, L, num_heads, d_k)
attn_values = attn_values.contiguous().view(batch_size, -1, d_model)  # (B, L, d_model)

print(attn_values.shape)

torch.Size([10, 20, 512])


In [18]:
outputs = w_0(attn_values)

print(outputs)
print(outputs.shape)

tensor([[[ 2.0647e-01,  1.7614e-02,  1.5888e-02,  ..., -7.6811e-02,
           4.7090e-02,  5.9217e-02],
         [ 1.8358e-01,  6.6134e-02,  4.2202e-02,  ..., -7.3667e-02,
           7.7152e-02,  7.9882e-02],
         [ 2.3217e-01,  7.2016e-02,  2.5100e-02,  ..., -1.0642e-01,
           6.3860e-02,  7.2267e-02],
         ...,
         [ 2.3534e-01,  5.0881e-02,  2.1790e-02,  ..., -7.7324e-02,
           5.9469e-02,  9.7691e-02],
         [ 2.3534e-01,  5.0881e-02,  2.1790e-02,  ..., -7.7324e-02,
           5.9469e-02,  9.7691e-02],
         [ 2.3534e-01,  5.0881e-02,  2.1790e-02,  ..., -7.7324e-02,
           5.9469e-02,  9.7691e-02]],

        [[ 3.5702e-01, -1.7859e-01,  3.9717e-02,  ...,  2.8024e-02,
          -3.8113e-02,  8.8643e-02],
         [ 3.9739e-01, -1.5972e-01,  7.6489e-02,  ...,  5.3090e-02,
          -6.2734e-06,  2.7271e-02],
         [ 4.0024e-01, -1.7384e-01,  5.6186e-02,  ...,  7.0403e-02,
          -8.3607e-03,  3.8790e-02],
         ...,
         [ 3.6427e-01, -1

#### 👨‍💻 <font color='green'><b>[ 코딩 ]</b></font> 위의 과정을 모두 합쳐 하나의 Multi-head attention 모듈을 구현해 봅시다.
```python
🐙
아래의 Multi-head attention 모듈에서 '#TODO'를 채워 모듈을 완성 시켜주세요.
위 실습에서 배운 내용이 큰 힌트가 될 거예요!
```

In [22]:
class MultiheadAttention(nn.Module):
  def __init__(self, dim_model, num_heads):
    super(MultiheadAttention, self).__init__()
    assert dim_model % num_heads == 0

    self.dim_model = dim_model
    self.num_heads = num_heads
    self.d_k = dim_model // num_heads

    self.w_q = nn.Linear(dim_model, dim_model)
    self.w_k = nn.Linear(dim_model, dim_model)
    self.w_v = nn.Linear(dim_model, dim_model)
    self.w_0 = nn.Linear(dim_model, dim_model)


  def forward(self, query, key, value):
    q = self.w_q(query)
    k = self.w_k(key)
    v = self.w_v(value)

    attn_values = self.self_attention(q,k,v)

    outputs = attn_values.view(-1, query.size(1), query.size(2))
    outputs = self.w_0(outputs)
    return outputs


  def self_attention(self,q,k,v):
      scores = torch.matmul(q, k.transpose(-2, -1))/math.sqrt(d_k) # (B,n_head,L,L)
      attn_dists = F.softmax(scores,dim=-1)
      output = torch.matmul(attn_dists, v).transpose(1, 2).contiguous().view(q.shape[0], -1, d_model)

      return output

In [ ]:
dim_model = 512
num_heads = 8
multihead_attn = MultiheadAttention(dim_model, num_heads)

outputs = multihead_attn(batch_emb, batch_emb, batch_emb)  # (B, L, d_model)

In [23]:
# 아래 코드는 수정하실 필요가 없습니다!
if outputs.shape == batch_emb.shape:
    print("🎉🎉🎉 성공!!! 🎉🎉🎉")
else:
    print("🐙 다시 도전해봐요!")

🎉🎉🎉 성공!!! 🎉🎉🎉
